# Learning in Multiagent Systems

In this lab we will study different learning approaches in the context of multiagent systems. We will start by implementing learning methods in the context of normal form games. Then we will study different algorithms in the context of Markov games.

Best reference for this topic is chapter 6 of https://www.marl-book.com/



# Installation

For this lab we will only need numpy. The toolbox nashpy has already several algorithms implemented and allows to verify some of the results.

## nashpy toolbox
Knight, V., & Campbell, J. (2018). Nashpy: A Python library for the computation of Nash equilibria. Journal of Open Source Software, 3(30), 904.
https://nashpy.readthedocs.io/en/stable/text-book/index.html
https://nashpy.readthedocs.io/en/stable/


In [ ]:
!pip install nashpy
import nashpy as nash
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
clear_output(wait=False)

# Repeated Games

We will start by defining several standard normal form games in Nashpy toolbox.
This toolbox already implements several algorithms to find Nash Equilibria and we can use it to verify the solutions for later examples.

In [ ]:
# @title Define Games

import warnings

Games = {}
A = np.array([[1, -1], [-1, 1]])
Games['Matching pennis'] = nash.Game(A)

A = np.array([[-1, -9], [0,-6]])
B = np.array([[-1, 0], [-9,-6]])
Games['Prisioner dilema'] = nash.Game(A,B)

A = np.array([[-2, 6], [0, 3]])
B = np.array([[-2, 0], [6, 3]])
Games['Hawk Dove'] = nash.Game(A,B)

A = np.array([[0,-1,1],[1,0,-1],[-1,1,0]])
Games['Rock Paper Scissor'] = nash.Game(A)

A = np.array([[10,1],[8,5]])
B = np.array([[10,8],[1,5]])
Games['Stag Hunt'] = nash.Game(A,B)


In [ ]:
# @title Auxiliary functions

# get value of a play in a nashpy game
def get_value(g,actA,actB):
    return g.payoff_matrices[0][actA,actB],g.payoff_matrices[1][actA,actB]

# egreedy function
# e is the probability of choosing the best action
def egreedy(v,e=0.95):
    NA = len(v)
    b = np.isclose(v,np.max(v))
    no = np.sum(b)
    if no<NA:
        p = b*e/no+(1-b)*(1-e)/(NA-no)
    else:
        p = b/no

    return int(np.random.choice(np.arange(NA),p=p))


def plothistofplay(Logn):
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')
    hist, xedges, yedges = np.histogram2d(Logn[:,0], Logn[:,1], bins=(np.max(Logn[:,0:2])+1), range=[[-0.5, np.max(Logn[:,0:2])+0.5], [-0.5, np.max(Logn[:,0:2])+0.5]])

    # Construct arrays for the anchor positions of the 16 bars.
    xpos, ypos = np.meshgrid(xedges[:-1] + 0.25, yedges[:-1] + 0.25, indexing="ij")
    xpos = xpos.ravel()
    ypos = ypos.ravel()
    zpos = 0

    # Construct arrays with the dimensions for the 16 bars.
    dx = dy = 0.5 * np.ones_like(zpos)
    dz = hist.ravel()

    ax.bar3d(xpos, ypos, zpos, dx, dy, dz, zsort='average')
    ax.set_xlabel('row player', fontsize=12, rotation=0)
    ax.set_ylabel('column player', fontsize=12, rotation=150)
    ax.set_xticks(np.arange(1+np.max(Logn[:,0:2])))
    ax.set_yticks(np.arange(1+np.max(Logn[:,0:2])))

    plt.show()


In [ ]:
# @title Game solutions based on Nashpy

for kk in Games.keys():
    g = Games[kk]

    if g.zero_sum:
        a = g.linear_program()
    else:
        a = g.vertex_enumeration()
        #b = g.support_enumeration()
    print(kk,"\n Nash equilibria\n",list(a))

    # Ficticious Play
    np.random.seed(0)
    iterations = 5000
    play_counts = g.fictitious_play(iterations=iterations)
    play_counts=np.array(list(play_counts))
    print(" ficticious play\n", play_counts[-1,:,:]/(play_counts.shape[0]-1))

Useful links:
*   https://nashpy.readthedocs.io/en/stable/text-book/normal-form-games.html
*   https://nashpy.readthedocs.io/en/stable/text-book/zero-sum-games.html
*   https://nashpy.readthedocs.io/en/stable/text-book/vertex-enumeration.html





# Repeated Games

Consider the previous normal form games as a repeated game, we will implement 2 different approaches:
* independent learning (refer to Sec. 5.3.2 https://www.marl-book.com/);
* ficticious play (refer to Sec. 6.3.1 https://www.marl-book.com/);

In most cases you can use the Q-Learning approach to compute the value of a given action:

$Q(s_t,a_t) = Q(s_t,a_t) + \alpha (r + \gamma \max_b Q(s_{t+1},b) - Q(s_t,a_t))$

in the case of normal form games we do not have state and so we can use:

$Q(a_t) = Q(a_t) + \alpha (r - Q(a_t))$

## Independent Learning
In independent learning the agents are unaware of other agents in the environment.

In this case what information about the environment are they using?

## Ficticious Play
In ficticious play the agent learn a model of the behaviour of the other agents and then play the best response to that model.
In this case what information about the environment are they using?
How can they learn the model of the behavior?


In [ ]:
# @title Independent Learning
# Independent learners are unaware of the oponents, in practice the oponnents are part of the environment.

for kk in Games.keys():#['Prisioner dilema']:#
    # Select a game.
    g = Games[kk]
    # Get the number of actions for each player.
    n0, n1 = g.payoff_matrices[0].shape

    # Start by defining a Q function for each agent. What should be its size?
    Q=[[],[]]
    # TODO
    # Q[0] =
    # Q[1] =

    # Lets create a log to track the learning progress and check if an equilibria is reached.
    Log = []
    A = [-1,-1]
    for ii in range(5000):
        # Select action for each agent (use an $\epsilon-greedy$ stretegy).
        # TODO
        # A[0] =
        # A[1] =
        r = get_value(g,A[0],A[1])
        Log.append([A[0],A[1],r[0],r[1]])

        # Update the estimate of the value Q-function for each agent (you can use a learning rate of 0.1).
        for aa in range(2):
            # TODO
            # Q[aa] =

    if g.zero_sum:
        a = g.linear_program()
    else:
        a = g.vertex_enumeration()

    print(kk,"\n Nash equilibria\n",list(a))
    print("Q\n",Q[0],"\n",Q[1])
    plothistofplay(np.array(Log))


In [ ]:
# @title Ficticious play
# In Ficticious play the learners are aware of the oponents.

def best_response(A,B):
    a = np.argmax(A)
    b = np.argmax(B)
    return a,b

for kk in Games.keys():
    # Select a game.
    g = Games[kk]

    # Get the number of actions for each player.
    n0, n1 = g.payoff_matrices[0].shape

    C = np.zeros((n0,n1))+1e-6

    # Lets create a log to track the learning progress and check if an equilibria is reached.
    Log = []
    A = [-1,-1]
    for ii in range(5000):

        # Select action for each agent.
        # TODO
        # policyA =
        # policyB =
        # A[0] =
        # A[1] =

        C[A[0],A[1]] += 1

        r = get_value(g,A[0],A[1])
        Log.append([A[0],A[1],r[0],r[1]])

    if g.zero_sum:
        a = g.linear_program()
    else:
        a = g.vertex_enumeration()

    print(kk,"\n Nash equilibria\n",list(a))
    plothistofplay(np.array(Log))

Question

* Compare the behavior of ficticious play and independent learning.
* Can we find Nash equilibria in games using learning algorithms?
* When does Q-learning converge to Nash equilibria?
* What does ficticious play assume? Can we do even better?

# Markov Games

In this section we will expand the learning approaches seen in repeated games to Markov Games, i.e. games that have a state that changes with the actions made by the agents. In the general, the transitions can be stochastic, but for simplicity we will assume deterministic transitions.

We will consider a Stag-Hunt game where two agents can either capture a Stag or a Hare. In this simplified problem the environment is just a corridor with 6 cells. The actions are 0,1,2 corresponding to moving (left, stop, right). The rules of the game are as follows:
* the reward is 0 at any step except if either the stag or the hare are captured
* the hare can be captured by any agent individually (the agent that captures receives .8 the other .1)
* the hare can be captured by the two agents simultaneously (both receive .5)
* the stag needs the two agents to capture it simultaneously (both receive 1.)
* if either agent occupies the same cell as the hare or the stag they make noise and the preys escape ending the episode

The state is represented as a vector of dimension four as follows [agent0-position, agent1-position, stag-position, hare-position]. The following situation would correspond to the state x=[3,3,0,5]

 \begin{pmatrix}
  |S|& |& |&A_0, A_1 |& |&H|
 \end{pmatrix}

 step(x,[0,2]) would result in state x = [2,4,0,5]
 \begin{pmatrix}
  |S|& |&A_0 |& |&A_1 |&H|
 \end{pmatrix}


In [ ]:
# @title
import numpy as np
#https://en.wikipedia.org/wiki/Stag_hunt

#Stag hunt environment
# we consider it as a corridor with NL cells
NL = 6
# state is a vector of dimension four as follows [agent0-position, agent1-position, stag-position, hare-position]
# the actions are 0,1,2 corresponding to moving left, stop, right
# the reward is 0 at any step except if either the stag or the hare are captured
# the hare can be captured by any agent individually
# the stag needs the two agents to capture it simultaneously
# if either agent occupies the same cell as the hare or the stag they make noise and the preys escape ending the episode


# the function step receives a state and a vector of actions and returns the new state and the reward for each agent
# if the location of either the stag of the hare is -1 is means they escapted and the episode should finish
def step(x,A):
    nx = x[:] # state is [agent0 position, agent1 position, stag position, hare position]
    nx[0] = int(np.clip(x[0] + (A[0]-1),0, NL-1))
    nx[1] = int(np.clip(x[1] + (A[1]-1),0, NL-1))
    r = [0.,0.]
    if (nx[0]==nx[3]) & (nx[1]==nx[3]):
        r[0] = .5
        r[1] = .5
        nx[3] = -1
        nx[2] = -1
    elif (nx[0]==nx[2]) & (nx[1]==nx[2]):
        r[0] = 1
        r[1] = 1
        nx[3] = -1
        nx[2] = -1
    elif (nx[0]==nx[2]) | (nx[1]==nx[2]):
        r[0] = 0
        r[1] = 0
        nx[3] = -1
        nx[2] = -1
    elif (nx[0]==nx[3]) & (nx[1]!=nx[3]):
        r[0] = .8
        r[1] = .1
        nx[3] = -1
        nx[2] = -1
    elif (nx[0]!=nx[3]) & (nx[1]==nx[3]):
        r[0] = .1
        r[1] = .8
        nx[3] = -1
        nx[2] = -1
    else:
        r = [0, 0]

    return nx, r

# Questions

* Assuming that the agents start in cell 3 what is the optimal policy for different values of $\gamma$?
Compute a normal-form game considering only the actions going left and going right (until the episode ends).

* Assuming that the agent 0 starts in cell 1, for $\gamma=0.9$ on which cell does agent 1 needs to start to prefer going left or going right?

* Assuming that the episode does not end after capturing the hare. Discuss what might happen.

# Class definition for the agents

This class implements:
* independent learners with partial state (unaware of others, refer to Sec. 5.3.2 https://www.marl-book.com/);
* independent learners with full state (aware of others);
* centralized learning (assuming the sum of rewards, refer to Sec. 5.3.1 https://www.marl-book.com/);
* joint-action learners with agent modelling (an extension of ficticious play for markov games, refer to Sec. 6.3.2 https://www.marl-book.com/);


Questions:
* What is the state for each type of agent?
* What and how are they learning about the environment and agents?
* How do they select the actions to make?


In [ ]:
# @title Class definition for the agents
class Agent:
    def __init__(self, NS, NA, id, NL = NL, alpha = 0.1, gamma = 0.9, agentType = 'independent'):
        self.alpha = alpha
        self.gamma = gamma
        self.id = id
        self.agentType = agentType
        self.NL = NL
        self.NA = NA

        if self.agentType == 'independent':
            self.Q = np.ones((NL,NA))*2
        elif self.agentType == 'observ':
            self.Q = np.ones((NL*NL,NA))*2
        elif self.agentType == 'central':
            self.Q = np.ones((NL*NL,NA*NA))*2
        elif self.agentType == 'JALAM':
            self.Q = np.ones((NL*NL,NA,NA))*2
            self.C = np.zeros((NL*NL,NA,NA))+0.001

    def __str__(self):

        return f"#{self.id} Qshape{self.Q.shape}"

    # this function takes the whole environment state and projects it in the state that each type of agent has access
    def observ2state(self, x):
        if self.agentType == 'independent':
            return x[self.id]
        elif self.agentType == 'observ':
            return np.ravel_multi_index(x[0:2],[self.NL,self.NL])
        elif self.agentType == 'central':
            return np.ravel_multi_index(x[0:2],[self.NL,self.NL])
        elif self.agentType == 'JALAM':
            return np.ravel_multi_index(x[0:2],[self.NL,self.NL])

    # this function is the learning update after any iteration with the environment, it gets
    # x current state
    # nx next state
    # a selected actions
    # r rewards obtained
    def update(self,x,nx,a,r):
        xi = self.observ2state(x)
        nxi = self.observ2state(nx)

        if self.agentType == 'central':
            ai = np.ravel_multi_index(a,[self.NA,self.NA])
            self.Q[xi,ai] += self.alpha * (np.sum(r) + self.gamma * np.max(self.Q[nxi,:]) - self.Q[xi,ai])
        elif self.agentType == 'JALAM':
            self.C[xi,a[self.id],a[1-self.id]] += 1
            self.Q[xi,a[self.id],a[1-self.id]] += self.alpha * (r[self.id] + self.gamma * np.max(self.Q[nxi,:]) - self.Q[xi,a[self.id],a[1-self.id]])
        else:
            self.Q[xi,a[self.id]] += self.alpha * (r[self.id] + self.gamma * np.max(self.Q[nxi,:]) - self.Q[xi,a[self.id]])

        return self.Q[x,:]

    # choosing the action to make in a given state x
    def chooseAction(self,x,e):
        xi = self.observ2state(x)
        if type(e) is float:
            if self.agentType == 'central':
                A = egreedy(self.Q[xi,:],e=e)
                a = np.unravel_index(A,[self.NA,self.NA])
                return a[self.id]
            elif self.agentType == 'JALAM':
                policyB = np.sum(self.C[xi,:,:],axis = 1)
                policyB = policyB/np.sum(policyB)

                bestreponseA = self.Q[xi,:,:] @ policyB.T

                actA = np.argmax(bestreponseA)
                return actA
            else:
                return egreedy(self.Q[xi,:],e=e)


# tun the algorithm for N steps
# x0 - if an initial state is provided the agents always start there, if not a random state is always generate after one prey escapes
def run(N,x0=[],e = 0.9,gamma = 0.9, agentType = 'independent'):

    if x0==[]:
        x = [np.random.randint(1,5),np.random.randint(1,5),0,5]
    else:
        x = x0[:]
    A = np.ones(2,dtype = int)
    Log = []
    Ag = [Agent(6,3,0,gamma = gamma,agentType = agentType), Agent(6,3,1,gamma = gamma,agentType = agentType)]

    for ii in range(0,N):
        A[0] = Ag[0].chooseAction(x, e)
        A[1] = Ag[1].chooseAction(x, e)
        nx,r = step(x,A)

        Ag[0].update(x,nx,A,r)
        Ag[1].update(x,nx,A,r)

        #print(A-1,nx,r)
        x = nx
        if (x[2]==-1)|(x[3]==-1):
            Log += [np.max(r)]
            if x0==[]:
                x = [np.random.randint(1,5),np.random.randint(1,5),0,5]
            else:
                x = x0[:]

    return Ag, Log

# this function allows to verify the action choosing in all possible initial states to verify what have the agents learned
def test(Ag):

    A = np.ones(2,dtype = int)
    Log = np.zeros((NL,NL))
    for i1 in [1,2,3,4]:
        for i2 in [1,2,3,4]:
            x0=[i1,i2,0,5]

            x = x0[:]
            for ii in range(0,100):
                A[0] = Ag[0].chooseAction(x, e = 1.0)
                A[1] = Ag[1].chooseAction(x, e = 1.0)
                nx,r = step(x,A)

                #print(A-1,nx,r)
                x = nx
                if (x[2]==-1)|(x[3]==-1):
                    Log[i1,i2] = np.max(r)
                    break

    return Log[1:NL-1,1:NL-1]


In [ ]:
# @title Compare the behaviour of different learning methods in the Stag-Hunt Markov Game $\gamma=.95$
for agentType in ['independent', 'observ', 'central','JALAM']:
    print(agentType)
    # run the different methods for 20000 steps
    Ag, Log = run(20000, e = 0.9, gamma = 0.95, agentType = agentType)
    Log = test(Ag)
    print("Value if the agents start in the initial location given by the row-column in the matrix.\n The second row, third column correspond to the initial state [3,4,0,5]")
    print(Log)

In [ ]:
# @title Compare the behaviour of different learning methods in the Stag-Hunt Markov Game $\gamma=.75$
for agentType in ['independent', 'observ', 'central','JALAM']:
    print(agentType)
    # run the different methods for 20000 steps
    Ag, Log = run(20000, e = 0.9, gamma = 0.75, agentType = agentType)
    # test the equilibria for all possible starting positions of the agents
    Log = test(Ag)
    print("Value if the agents start in the initial location given by the row-column in the matrix.\n The second row, third column correspond to the initial state [3,4,0,5]")
    print(Log)

# Questions

* What is the impact of $\epsilon$ in the learning?
* What does it mean to have a centralized learning in this context? Can we have two rewards? Discuss.
* What is the impact of the initial location of the agents? In particular for the independent case.


# Bonus Questions
* Can you implement a version of minimaxQ (using the demo code provided)?
* Change the game to allow it to continue even after getting the hare. What new behaviors will show?

In [ ]:
# @title Bonus Question "MinimaxQ"
# If interested, you can replace the best response in Ficticious Play with the MinimaxQ approach, chapter 6.2 from the book.
import numpy as np
from scipy.optimize import linprog

def minimaxQ(Q):
    n0, n1 = 2, 2
    c = np.zeros(n0 + 1)
    c[0] = -1
    A_ub = np.ones((n1, n0 + 1))
    A_ub[:, 1:] = -Q[:n0, :n1].T
    b_ub = np.zeros(n1)
    A_eq = np.ones((1, n0 + 1))
    A_eq[0, 0] = 0
    b_eq = [1]
    bounds = ((None, None),) + ((0, 1),) * n0

    res = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=bounds)

    return res

# Q = np.array([[6,4],[2,5]])
# res = minimaxQ(Q)
# print(res.x[1:])